In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re

In [4]:
url = """https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/"""

In [5]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
soup=BeautifulSoup(page.content, 'html.parser')

In [7]:
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
title

'What Jobs Will Robots Take From Humans in The Future?'

In [8]:
content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
content

' Introduction AI is rapidly evolving in the employment sector, particularly in matters involving business and finance. Finance, management, economics, and accounting are now among the most popular university courses globally; particularly at the graduate level, due to their high employability. However, the evolution of machinery in industries is changing that. According to research, 230,000 jobs in these sectors may be replaced by AI agents in the next 5 years. This is due to the nature of the work, as employees are responsible for tasks such as data analysis and keeping track of numerical information; which machines excel at. Large, complicated data sets can be analyzed faster and more efficiently by AI-powered computers than by people. Algorithmic trading procedures that produce automated deals are less likely to be produced without minute errors when undertaken by humans. In such matters involving industrial work, a subsection of artificial intelligence is used; namely, machine lea

In [9]:
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
content

' Introduction AI is rapidly evolving in the employment sector particularly in matters involving business and finance Finance management economics and accounting are now among the most popular university courses globally particularly at the graduate level due to their high employability However the evolution of machinery in industries is changing that According to research 230000 jobs in these sectors may be replaced by AI agents in the next 5 years This is due to the nature of the work as employees are responsible for tasks such as data analysis and keeping track of numerical information which machines excel at Large complicated data sets can be analyzed faster and more efficiently by AIpowered computers than by people Algorithmic trading procedures that produce automated deals are less likely to be produced without minute errors when undertaken by humans In such matters involving industrial work a subsection of artificial intelligence is used namely machine learning  Machine learning

In [10]:
text = content.split()
text

['Introduction',
 'AI',
 'is',
 'rapidly',
 'evolving',
 'in',
 'the',
 'employment',
 'sector',
 'particularly',
 'in',
 'matters',
 'involving',
 'business',
 'and',
 'finance',
 'Finance',
 'management',
 'economics',
 'and',
 'accounting',
 'are',
 'now',
 'among',
 'the',
 'most',
 'popular',
 'university',
 'courses',
 'globally',
 'particularly',
 'at',
 'the',
 'graduate',
 'level',
 'due',
 'to',
 'their',
 'high',
 'employability',
 'However',
 'the',
 'evolution',
 'of',
 'machinery',
 'in',
 'industries',
 'is',
 'changing',
 'that',
 'According',
 'to',
 'research',
 '230000',
 'jobs',
 'in',
 'these',
 'sectors',
 'may',
 'be',
 'replaced',
 'by',
 'AI',
 'agents',
 'in',
 'the',
 'next',
 '5',
 'years',
 'This',
 'is',
 'due',
 'to',
 'the',
 'nature',
 'of',
 'the',
 'work',
 'as',
 'employees',
 'are',
 'responsible',
 'for',
 'tasks',
 'such',
 'as',
 'data',
 'analysis',
 'and',
 'keeping',
 'track',
 'of',
 'numerical',
 'information',
 'which',
 'machines',
 'excel

In [11]:
len(text)

1700

In [12]:
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]

In [13]:
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")

In [14]:
Positive_score=len(pos_count)
print(Positive_score)

73


In [15]:
#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]

In [16]:
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")

In [17]:
Negative_score=len(neg_count)
print(Negative_score)

33


In [18]:
filter_content = ' '.join(text)

In [19]:
data=[[url,title,content,filter_content,Positive_score,Negative_score]]

In [20]:
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])

In [21]:
from textblob import TextBlob

In [22]:
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])

# Adding Subjectivity & Polarity
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)

data

,url,title,content,filter_content,Positive_Score,Negative_Score,polarity,subjectivity
0,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction AI is rapidly evolving in the em...,Introduction AI is rapidly evolving in the emp...,73,33,0.11147,0.483201


In [23]:
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)

Word average = 9220.0


In [24]:
import textstat

In [25]:
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print(FOG_INDEX)

684.87


In [26]:
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
AVG_NUMBER_OF_WORDS_PER_SENTENCE=print(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))

1700.0


In [27]:
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print(COMPLEX_WORDS)

3340


In [28]:
#Word Count
Word_Count=len(content)
print(Word_Count)

10922


In [29]:
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print(pcw)

30.580479765610697


In [30]:
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print(Average_Word_Length)

5.423529411764706


In [31]:
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
print(syllable_count/len(content.split()))

The AVG number of syllables in the word is: 
2.1876470588235293


In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re
url = """https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/"""
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
soup=BeautifulSoup(page.content, 'html.parser')
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
# title

content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
# print(content)
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
# print(content)
text = content.split()
# print(text)
len(text)
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)


filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)
import textstat
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print("FOG INDEX = ",FOG_INDEX)
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
print("Average no of words per sentence")
AVG_NUMBER_OF_WORDS_PER_SENTENCE=(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))
print(AVG_NUMBER_OF_WORDS_PER_SENTENCE)
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print("Complex Words",COMPLEX_WORDS)
#Word Count
Word_Count=len(content)
print("Word Count",Word_Count)
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print("Percentage of Complex Words",pcw)
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print("Average Word per Length",Average_Word_Length)
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
syllable = (syllable_count/len(content.split()))
print(syllable)

data = [[url,title,content,filter_content,Positive_score,Negative_score,AVG_SENTENCE_LENGTH,pcw,FOG_INDEX,
         AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score","Avg_Sentence_Length"
                               ,"Percentage_Complex_Word","Fog_Index"," AVG_NUMBER_OF_WORDS_PER_SENTENCE","COMPLEX_WORDS",
                               "Word_Count","syllable","Average_Word_Length"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data

73
33
Word average = 9220.0
FOG INDEX =  684.87
Average no of words per sentence
1700.0
Complex Words 3340
Word Count 10922
Percentage of Complex Words 30.580479765610697
Average Word per Length 5.423529411764706
The AVG number of syllables in the word is: 
2.1876470588235293


,url,title,content,filter_content,Positive_Score,Negative_Score,Avg_Sentence_Length,Percentage_Complex_Word,Fog_Index,AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length,polarity,subjectivity
0,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,Introduction AI is rapidly evolving in the em...,Introduction AI is rapidly evolving in the emp...,73,33,9220.0,30.58048,684.87,1700.0,3340,10922,2.187647,5.423529,0.11147,0.483201


In [2]:
data.to_csv(r'C:\Users\Om Bhandwalkar\Desktop\BlackCoffer Assignment\Output\url_39.csv')